In [1]:
import os
import logging
import json
import re
import traceback
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain_experimental.tools import PythonREPLTool
from langchain.hub import pull as hub_pull
from PyPDF2 import PdfReader  

In [2]:
# Carrega variáveis de ambiente
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Verifica a chave da API
if not api_key:
    raise ValueError("OPENAI_API_KEY não encontrada nas variáveis de ambiente. Por favor, verifique seu arquivo .env.")

# Configuração de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
def inicializar_llm():
    """Inicializa o LLM com as configurações especificadas."""
    return ChatOpenAI(
        temperature=0.1,
        model="gpt-4o-mini",
        api_key=api_key,
        verbose=True
    )

def criar_agente_react(llm):
    """Cria um agente ReAct com cache de ferramentas e suporte à execução de Python."""
    try:
        modelo_prompt_react = hub_pull("hwchase17/react")
        tools = [PythonREPLTool()]
        agente = initialize_agent(
            tools,
            llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True,
            agent_kwargs={"prompt": modelo_prompt_react}
        )
        return agente
    except Exception as e:
        logger.error("Erro ao criar agente ReAct: %s", e)
        raise

def processar_texto(texto):
    """Processa o texto e retorna um JSON estruturado com tratamento de erros aprimorado."""
    modelo_prompt = """
    Você é um especialista em extração de informações estruturadas de currículos.
    Analise cuidadosamente o texto completo do CV e extraia os detalhes em um formato JSON estruturado e preciso.
    Priorize a clareza e a completude.

    TEXTO DO CV:
    {texto}

    INSTRUÇÕES PARA O OUTPUT:
    - Retorne um JSON completo com todos os campos esperados
    - Use um texto descritivo e conciso
    - Se faltar alguma informação, use strings ou listas vazias
    - Garanta formatação e legibilidade adequadas

    ESTRUTURA DE JSON EXIGIDA:
    {{
        "informacoes_pessoais": {{
            "nome": "Nome Completo",
            "cidade": "Cidade, Estado/País",
            "bairro": "Bairro Opcional",
            "email": "email@exemplo.com",
            "telefone": "Telefone Opcional",
            "cargo": "Cargo Atual ou Desejado"
        }},
        "resumo_qualificacoes": [{{
            "resumo": "Visão geral profissional breve",
            "qualificacoes_chave": [
                {{"qualificacao": "Habilidade ou realização importante"}},
                {{"qualificacao": "Outra habilidade importante"}}
            ]
        }}],
        "experiencia_profissional": [
            {{
                "empresa": "Nome da Empresa",
                "cargo": "Título do Cargo",
                "periodo": "Data de Início - Data de Término",
                "atividades": [
                    {{"atividade": "Responsabilidade ou realização chave"}},
                    {{"atividade": "Outra responsabilidade importante"}}
                ],
                "projetos": [
                    {{"titulo": "Nome do Projeto", "descricao": "Descrição do projeto"}}
                ]
            }}
        ],
        "educacao": [
            {{
                "instituicao": "Nome da Escola/Universidade",
                "grau": "Grau ou Certificação",
                "ano_formatura": "Ano"
            }}
        ],
        "certificacoes": [
            {{"certificado": "Nome da Certificação"}}
        ]
    }}
    """

    try:
        prompt = PromptTemplate(template=modelo_prompt, input_variables=["texto"])
        llm = ChatOpenAI(api_key=api_key, temperature=0, model="gpt-4o-mini")
        resultado = llm.invoke(prompt.format(texto=texto))

        print("Resposta Completa do Modelo:")
        print(resultado.content)

        dados_json = None
        try:
            dados_json = json.loads(resultado.content)
        except json.JSONDecodeError:
            correspondencia_json = re.search(r'\{.*\}', resultado.content, re.DOTALL | re.MULTILINE)
            if correspondencia_json:
                try:
                    dados_json = json.loads(correspondencia_json.group(0))
                except Exception as e:
                    print(f"Erro de extração JSON: {e}")

        estrutura_padrao = {
            "informacoes_pessoais": {
                "nome": "",
                "cidade": "",
                "email": "",
                "telefone": "",
                "cargo": ""
            },
            "resumo_qualificacoes": [],
            "experiencia_profissional": [],
            "educacao": [],
            "certificacoes": []
        }

        if dados_json:
            for chave in estrutura_padrao:
                if chave not in dados_json:
                    dados_json[chave] = estrutura_padrao[chave]

        return dados_json or estrutura_padrao

    except Exception as e:
        print("Erro detalhado ao processar texto:")
        print(traceback.format_exc())
        return estrutura_padrao

def limpar_texto(texto):
    """Limpa e normaliza o texto extraído."""
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'\n*Página \d+ de \d+\n*', '', texto)
    texto = re.sub(r'\n{3,}', '\n\n', texto)
    texto = texto.strip()
    return texto

def extrair_texto_de_pdf(caminho_pdf):
    """Extrai o texto de um arquivo PDF."""
    try:
        leitor_pdf = PdfReader(caminho_pdf)
        texto = "\n".join([pagina.extract_text() for pagina in leitor_pdf.pages])
        texto_limpo = limpar_texto(texto)
        return texto_limpo
    except Exception as e:
        print(f"Erro ao extrair texto do PDF: {e}")
        return ""

def main(caminho_pdf):
    """Função principal para processar PDF e gerar documento de currículo."""
    texto_pdf = extrair_texto_de_pdf(caminho_pdf)

    if not texto_pdf:
        print("Nenhum texto pôde ser extraído do PDF.")
        return

    dados_curriculo = processar_texto(texto_pdf)
    # Você pode agora usar o dicionário dados_curriculo para gerar um documento de currículo

In [4]:
dados_curriculo = main('Profile.pdf')
dados_curriculo

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Resposta Completa do Modelo:
```json
{
    "informacoes_pessoais": {
        "nome": "Pedro Lustosa",
        "cidade": "Sobral, Ceará, Brasil",
        "bairro": "",
        "email": "pedrolustosab@gmail.com",
        "telefone": "85996859064",
        "cargo": "Analista de Dados"
    },
    "resumo_qualificacoes": [{
        "resumo": "Como Analista de Dados apaixonado, minha jornada é guiada pela excelência e uma curiosidade insaciável. Vejo além dos números; eu os transformo em narrativas que revelam insights e direcionam estratégias.",
        "qualificacoes_chave": [
            {"qualificacao": "Expertise em Power BI, SQL e Python"},
            {"qualificacao": "Capacidade de criar soluções inovadoras"}
        ]
    }],
    "experiencia_profissional": [
        {
            "empresa": "Grupo Portfolio",
            "cargo": "Analista de BI",
            "periodo": "março de 2022 - Present",
            "atividades": [
                {"atividade": "Desenvolvimento de relatóri